In [4]:
import pandas as pd

log = pd.read_csv('log.csv', header=None)
log.columns = ['user_id','time', 'bet','win']
log
log.time.isna().value_counts()


False    985
True      15
Name: time, dtype: int64

In [5]:
log2 = log.dropna(axis=0)

log2.iloc[14].isna()

user_id    False
time       False
bet        False
win        False
Name: 384, dtype: bool

In [6]:
log3 = log.drop_duplicates(subset=['user_id', 'time'])
log3 = log3.dropna(axis=0, subset=['time'])

log3.time = log3.time.str.replace('[','')
log3.time.apply(pd.to_datetime)

0     2019-01-01 14:06:51
1     2019-01-01 14:51:16
2     2019-01-01 16:31:16
3     2019-01-01 17:17:51
4     2019-01-01 21:31:18
              ...        
991   2019-04-20 12:55:41
992   2019-04-20 14:59:36
993   2019-04-20 17:09:56
994   2019-04-20 18:10:07
995   2019-04-20 09:59:58
Name: time, Length: 985, dtype: datetime64[ns]

In [7]:
log = pd.read_csv("log.csv")
log.columns = ['user_id', 'time', 'bet', 'win']
log = log.dropna(axis=0, subset=['time'])
log['time'] = log['time'].apply(lambda x: x[1:])
log['time'] = pd.to_datetime(log['time'])
#log.dayofweek = log['time'].dt.dayofweek
#log.dayofweek[log.dayofweek >= 5].count()

log['time'].dt.dayofweek.apply(lambda x: 1 if x >=5 else 0).sum()

log['time'].dt.hour.apply(lambda x:
                               'night' if 0 <=x<=5 else
                               'morning' if 6 <=x<=11 else
                               'afternoon' if 12 <=x<=17 else
                               'evening').value_counts()

night        265
morning      253
afternoon    239
evening      227
Name: time, dtype: int64

In [8]:
log = pd.read_csv("log.csv", header=None)
log.columns = ['user_id', 'time', 'bet', 'win']
log = log.dropna(axis=0)
log['time'] = log['time'].apply(lambda x: x[1:])
log['time'] = pd.to_datetime(log['time'])
log['hour'] = log['time'].dt.hour

log

log = pd.read_csv("log.csv", header=None)
log.columns = ['user_id', 'time', 'bet', 'win']

log.bet.dropna().mean()

6785.738144329897

In [9]:
log = pd.read_csv("log.csv", header=None)
log.columns = ['user_id', 'time', 'bet', 'win']

log['bet'] = log['bet'].fillna(0)
log['win'] = log['win'].fillna(0)

log['bet'].value_counts()

def fillna_win(row):
    return row.win - row.bet

log['net'] = log.apply(lambda row: fillna_win(row), axis=1)

print('bet %', log.bet[log.bet > 0].count() / log.bet.count() * 100)
print('bet avg', log.bet[log.bet > 0].mean())
print('bet min', log.bet[log.bet > 0].min())
print('bet min count', log.bet[log.bet == 100].count())
print('net avg', log.net[log.bet > 0].mean())
print('net avg loss', log.net[log.net < 0].mean())
print('net % loss', log.net[log.net < 0].count()/ log.net.count() * 100)
print('net % win', log.net[log.net >= 0].count()/ log.net.count() * 100)

#log.net[log.net >0].mean()

bet % 48.5
bet avg 6785.738144329897
bet min 100.0
bet min count 48
net avg 20421.892783505155
net avg loss -3372.743515850144
net % loss 34.699999999999996
net % win 65.3


In [10]:
log = pd.read_csv("log.csv", header=None)
log.columns = ['user_id', 'time', 'bet', 'win']

log = log.dropna(axis=0, subset=['time'])
log['bet'] = log['bet'].fillna(0)
log['win'] = log['win'].fillna(0)

log.time = log.time.str.replace('[','')
log.time = log.time.apply(pd.to_datetime)


def fillna_win(row):
    return row.win - row.bet

log['net'] = log.apply(lambda row: fillna_win(row), axis=1)

users = pd.read_csv('users.csv', delimiter='\t', encoding='koi8_r')
users.columns = ['user_id', 'email', 'geo']
#users.info()

users.user_id = users.user_id.apply(lambda x: x.lower())

log = log[log.user_id != '#error']
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])

df = pd.merge(log, users, on='user_id')

#df.info()

df.groupby('user_id').net.sum().median()

df_grouped = pd.DataFrame()

df_grouped['bet_sum'] = df.groupby('user_id').bet.sum()
df_grouped['bet_attends'] = df.query('bet == 0').groupby('user_id').bet.count()

df_grouped.query('bet_sum > 0').mean()

bet_sum        32122.05
bet_attends        5.05
dtype: float64

In [11]:
df_grouped['bet_first_attend'] = df.groupby('user_id').time.min()
df_grouped['bet_first_pay'] = df.query('bet > 0').groupby('user_id').time.min()

#df_grouped['bet_first_attend']

df_grouped['bet_interval'] = df_grouped.query('bet_sum > 0')['bet_first_pay'] \
                             - df_grouped.query('bet_sum > 0')['bet_first_attend']

df_grouped['bet_interval'].mean()

Timedelta('47 days 01:34:33.930000')

In [15]:
df.groupby('geo').win.sum().sort_values()

geo
Тюмень                 4701.0
Екатеринбург          36682.0
Ставрополь            46003.0
Краснодар             62718.0
Хабаровск             65459.0
Пермь                 67734.0
Арзангелтск           74375.0
Красноярск            84767.0
Ижевск                84895.0
Ярославль             97441.0
Казань                97806.0
Санкт-Петербург      151007.0
Воронеж              184338.0
Москва             11959741.0
Name: win, dtype: float64

In [21]:
df_geo = pd.DataFrame()
df_geo['mean'] = df.query('bet > 0').groupby('geo').bet.mean()
df_geo['mean'].max() / df_geo['mean'].min()


127.81602335164835

In [24]:
log = pd.read_csv("log.csv", header=None)
log.columns = ['user_id', 'time', 'bet', 'win']

log = log.dropna(axis=0, subset=['time'])
log['bet'] = log['bet'].fillna(0)
log['win'] = log['win'].fillna(0)

log.time = log.time.str.replace('[','')
log.time = log.time.apply(pd.to_datetime)


def fillna_win(row):
    return row.win - row.bet

log['net'] = log.apply(lambda row: fillna_win(row), axis=1)

users = pd.read_csv('users.csv', delimiter='\t', encoding='koi8_r')
users.columns = ['user_id', 'email', 'geo']
#users.info()

users.user_id = users.user_id.apply(lambda x: x.lower())

log = log[log.user_id != '#error']
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])

df = pd.merge(log, users, on='user_id')

sample2 = df.groupby('geo').user_id.count()

sample2

geo
Арзангелтск         96
Воронеж             88
Екатеринбург        49
Ижевск              61
Казань              66
Краснодар           86
Красноярск          56
Москва              61
Пермь               55
Санкт-Петербург    115
Ставрополь          36
Тюмень              32
Хабаровск           60
Ярославль           89
Name: user_id, dtype: int64

TypeError: sequence item 0: expected str instance, int found